In [2]:
import pandas as pd
import numpy as np

<br>

<ul style="list-style-type:disc;">
  <li style = "font-family: Gulim; font-size:24px;line-height:1.4">
      <b> 전처리_시군구별 전체인구 및 도시인구비율<sup>1</sup> </b>
        <ul style = "list-style-type:circle;">
            <li style = "font-family: Gulim;font-size:16px;line-height:1.5"> 도시계획현황, 통계청<sup>1</sup> </li>
            <li style = "font-family: Gulim;font-size:16px;line-height:1.5"> 시각화 목적에 맞게 지역 ID를 생성한다.</li>

In [11]:
city = pd.read_csv("시군구별 전체인구 및 도시인구비율.csv", encoding='euc-kr')
city = city.rename(columns={'소재지(시군구)별(1)':'시도', 
                     '소재지(시군구)별(2)':'군구',
                     '전체인구(A) (명)':'전체인구',
                     '도시지역 인구비율 (%)':'도시인구비율'})
city.head()

,시도,군구,전체인구,도시인구비율
0,서울특별시,종로구,153065,100.0
1,서울특별시,중구,125725,100.0
2,서울특별시,용산구,228999,100.0
3,서울특별시,성동구,308221,100.0
4,서울특별시,광진구,355559,100.0


In [14]:
city_1 = city.copy()
for i in range(len(city_1)):
    if len(city_1.loc[i,'시도']) >= 3:
        if city_1.loc[i,'시도'][2:] in ['시','광역시', '특별시', '도']:
            city_1.loc[i,'시도'] = city_1.loc[i,'시도'][:2] 
        elif city_1.loc[i,'시도'][3]  == '도':
            city_1.loc[i,'시도'] = city_1.loc[i,'시도'][0] + city_1.loc[i,'시도'][2]     
    if city_1.loc[i,'군구'] == '고성군':
        if city_1.loc[i,'시도'] == '강원':
            city_1.loc[i,'군구'] = '고성(강원)'
        else:
            city_1.loc[i,'군구'] = '고성(경남)'  
    elif city_1.loc[i,'시도'] == '세종특별자치시':
        city_1.loc[i,'군구'] = '세종'
    else:
        if len(city_1.loc[i,'군구']) >= 4:
            city_1.loc[i,'군구'] = city_1.loc[i,'군구'][:3]

city_1.head()


,시도,군구,전체인구,도시인구비율
0,서울,종로구,153065,100.0
1,서울,중구,125725,100.0
2,서울,용산구,228999,100.0
3,서울,성동구,308221,100.0
4,서울,광진구,355559,100.0


In [19]:
special_cities =  ['서울',  '인천',  '대전', '대구', '울산', '부산', '광주']
over3_gungu = ['고성(강원)', '고성(경남)','동두천', '의정부', '일산동', '남양주', '일산서', '서대문', '동대문', '영등포', '부산진', '해운대', '서귀포']

In [21]:
tot_pop = city_1['전체인구'].sum()

In [23]:
city_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 4 columns):
시도        229 non-null object
군구        229 non-null object
전체인구      229 non-null int64
도시인구비율    229 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.2+ KB


In [29]:
city_2 = pd.DataFrame(columns=['ID', '전국대비인구비율', '지역별도시인구비율'])

n = len(city_1)
for i in range(n):
    # 군구 중 원래 3음절이 아닌 경우 맨 끝의 '구' 또는 '시' 를 떼어낸다
    if city_1.loc[i, '군구'] not in over3_gungu and len(str(city_1.loc[i, '군구'])) >= 3:
        city_1.loc[i,'군구'] = str(city_1.loc[i, '군구'])[0:-1]
    # 시도 가 특별시, 광역시 등일 경우 draw_korea 와 호환을 위해 ID 에 포함한다.
    if city_1.loc[i, '시도'] in special_cities:
        city_2.loc[i, 'ID'] = city_1.loc[i, '시도'] +' '+ str(city_1.loc[i, '군구'])
    else:
        city_2.loc[i, 'ID'] = city_1.loc[i, '군구']
    city_2.loc[i, '전국대비인구비율'] = city_1.loc[i,'전체인구']/tot_pop
    city_2.loc[i, '지역별도시인구비율'] = city_1.loc[i,'도시인구비율']
    
city_2.tail()

,ID,전국대비인구비율,지역별도시인구비율
224,함양,0.000772661,47.35
225,거창,0.00120509,65.47
226,합천,0.000885964,57.72
227,제주,0.00937648,92.16
228,서귀포,0.00349718,86.63


In [27]:
city_2.to_csv("data_soomin/시군구별인구비율.csv", encoding = 'utf-8-sig')
city_2.to_csv("data_soomin/시군구별인구비율_euckr인코딩.csv")

---------------------
<p style = "font-family:Gulim; font-size:26px;font-weight:bold">참조 링크 </p>
<br>


1. [도시지역 인구현황(시군구)](http://kosis.kr/search/search.do),한국토지주택공사,「도시계획현황」